In [ ]:
from sshtunnel import SSHTunnelForwarder
import psycopg2
import os
import pandas as pd

# SSH connection details.
ssh_host = 'loki.research.cs.dal.ca'
ssh_port = 22
ssh_username = os.getenv('LOKI_USERNAME')
ssh_password = os.getenv('LOKI_PASSWORD')

# Database connection details.
db_host = '127.0.0.1'
db_port = 5432
db_name = 'staging_db'
db_user = os.getenv('STAGING_DB_USERNAME')
db_password = os.getenv('STAGING_DB_PASSWORD')

# Establish SSH tunnel and connect to PostgreSQL.
try:
    with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_username,
        ssh_password=ssh_password,
        remote_bind_address=('127.0.0.1', db_port)  # Forwarding PostgreSQL port.
    ) as tunnel:
    
        # Connect to PostgreSQL database through the SSH tunnel.
        conn = psycopg2.connect(
            host=db_host,
            port=tunnel.local_bind_port,  # use the local port set by the tunnel.
            dbname=db_name,
            user=db_user,
            password=db_password
        )
        
        print("Database connection established")
        
        activity_query = "SELECT * FROM study_prositvd.activity;"
        df_activity = pd.read_sql_query(activity_query, conn)
        
        # Close database connection.
        conn.close()

except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
df_activity.head()

In [ ]:
df_activity['is_stationary']        = df_activity['value0'].apply(lambda x: int(x.split(', ')[1]))
df_activity['is_walking']           = df_activity['value0'].apply(lambda x: int(x.split(', ')[3]))
df_activity['is_running']           = df_activity['value0'].apply(lambda x: int(x.split(', ')[5]))
df_activity['is_in_vehicle']        = df_activity['value0'].apply(lambda x: int(x.split(', ')[7]))
df_activity['is_cycling']           = df_activity['value0'].apply(lambda x: int(x.split(', ')[9]))
df_activity['is_activity_unknown']  = df_activity['value0'].apply(lambda x: int(x.split(', ')[11]))
df_activity['confidence']           = df_activity['value0'].apply(lambda x: 1 if x.split(', ')[13] == 'High' else 0)

In [ ]:
df_activity.head()

In [ ]:
df_activity.drop('_id', axis=1, inplace=True)
df_activity.drop('value0', axis=1, inplace=True)

In [ ]:
df_activity.head()

In [60]:
# Filter out rows with low confidence and unknown activities.
df_known_confident = df_activity[(df_activity.is_activity_unknown == 0) & (df_activity.confidence == 1)]

In [61]:
print(f"Stationary count: {df_known_confident[(df_known_confident.is_stationary == 0)].shape[0]}")
print(f"Non-stationary count: {df_known_confident[(df_known_confident.is_stationary == 1)].shape[0]}")

Stationary count: 266028
Non-stationary count: 595798


In [62]:
participant_ids = df_known_confident['participantid'].drop_duplicates().to_list()

In [ ]:
# Function to calculate total duration from a list of windows.
def get_total_duration(start, end, windows):
    total_duration = timedelta(seconds=0)
    for window in windows:
        window_start, window_end = window
        if window_end >= start and window_start <= end:
            overlap_start = max(window_start, start)
            overlap_end = min(window_end, end)
            total_duration += overlap_end - overlap_start
    return total_duration

In [65]:
import datetime
from datetime import timedelta

In [66]:
# Define the start and end timestamps (observation window)
start_timestamp = pd.Timestamp('22:00:00').time()
end_timestamp = pd.Timestamp('08:00:00').time()
ascribe_duration_to_prev_day = True  # I.e. the day before wakeup. 

# Define the inactive threshold
inactive_threshold = timedelta(minutes=15)

In [72]:
all_results = {}    # participant_id : participant_results_df

for participant_id in participant_ids:
    participant_df = df_known_confident[df_known_confident.participantid == participant_id]
    
    participant_df.drop('uploadedat', axis=1, inplace=True)
    
    participant_df['is_active'] = (participant_df['is_stationary'].apply(lambda x: x == 1) & participant_df['is_in_vehicle'].apply(lambda x: x == 0)).apply(lambda x: 0 if x else 1)
    
    participant_df.drop('is_stationary', axis=1, inplace=True)
    participant_df.drop('is_walking', axis=1, inplace=True)
    participant_df.drop('is_running', axis=1, inplace=True)
    participant_df.drop('is_in_vehicle', axis=1, inplace=True)
    participant_df.drop('is_cycling', axis=1, inplace=True)
    participant_df.drop('is_activity_unknown', axis=1, inplace=True)
    participant_df.drop('confidence', axis=1, inplace=True)
    
    participant_df.drop('participantid', axis=1, inplace=True)
    
    # Extract sorted timestamps where `is_active` is 1.
    active_timestamps = participant_df[participant_df['is_active'] == 1].sort_values(by='measuredat')['measuredat'].tolist()

    ######################################################################
    # Extract active windows                                             #
    ######################################################################
    
    # Initialize empty list to store active windows
    active_windows = []
    
    # Initialize the first window
    start_time = active_timestamps[0]
    end_time = active_timestamps[0]
    
    # Iterate through the timestamps
    for i in range(1, len(active_timestamps)):
        current_time = active_timestamps[i]
        previous_time = active_timestamps[i - 1]
        
        # If the difference is greater than the inactive threshold, close the current window
        if current_time - previous_time > inactive_threshold:
            active_windows.append((start_time, end_time))
            start_time = current_time  # Start a new window
        
        end_time = current_time  # Update end time for the current window
    
    # Append the last window
    active_windows.append((start_time, end_time))
    
    
    ######################################################################
    # Extract inactive windows                                           #
    ######################################################################
    
    # Create the list of inactive windows
    inactive_windows = []
    
    # Start with the first inactive window, which starts at the end of the first active window
    prev_end = active_windows[0][1]
    
    for window in active_windows[1:]:
        current_start = window[0]
        
        # If there’s a gap between two active windows, it becomes an inactive window
        if current_start - prev_end > timedelta(seconds=0):
            inactive_windows.append((prev_end, current_start))
        
        # Update previous window end
        prev_end = window[1]
        
    # If there are no inactive windows, move on to the next participant.
    if not inactive_windows:
        continue
    
    # Create a dataframe for each participant.
    participant_results_df = pd.DataFrame(columns=['date', 'duration'])
    
    # Iterate from start date to end date for this participant.
    start_date = inactive_windows[0][0].date()
    end_date = inactive_windows[-1][-1].date()
    current_date = start_date
    while current_date <= end_date:
        
        # Construct full timestamps.
        if start_timestamp < end_timestamp:
            start = pd.to_datetime(f"{current_date} {start_timestamp}")
            end = pd.to_datetime(f"{current_date} {end_timestamp}")
        elif start_timestamp > end_timestamp:
            start = pd.to_datetime(f"{current_date} {start_timestamp}")
            end = pd.to_datetime(f"{current_date + timedelta(days=1)} {end_timestamp}")
        else:
            print("Error: invalid timestamp")
        
        # Compute ascribed date and duration of inactive minutes for that day.
        if ascribe_duration_to_prev_day:
            date = end.date() - timedelta(days=1)
        else:
            date = end.date()
        
        duration = get_total_duration(start, end, active_windows).total_seconds() / 60
        
        # Construct the row for the day.
        entry = { 'date': date, 'duration': duration }
        participant_results_df = pd.concat([participant_results_df, pd.DataFrame([entry])], ignore_index=True)
        
        current_date += timedelta(days=1)
    
    # Add participants result to the list of results df.
    all_results[participant_id] = participant_results_df

In [73]:
all_results.keys()

dict_keys(['prositvd5', 'prositvd6', 'prositvd9', 'prositvd11', 'prositvd15', 'prositvd17', 'prositvd24', 'prositvd26', 'prositvd30', 'prositvd31', 'prositvd34', 'prositvd39', 'prositvd41', 'prositvd42', 'prositvd45', 'prositvd49', 'prositvd61', 'prositvd65', 'prositvd68', 'prositvd78', 'prositvd80', 'prositvd87', 'prositvd88', 'prositvd89', 'prositvd90', 'prositvd91', 'prositvd92', 'prositvd96', 'prositvd97', 'prositvd105', 'prositvd106', 'prositvd109', 'prositvd110', 'prositvd113', 'prositvd117', 'prositvd119', 'prositvd121', 'prositvd123', 'prositvd128', 'prositvd129', 'prositvd155', 'prositvd164', 'prositvd180', 'prositvd181', 'prositvd183', 'prositvd185', 'prositvd186', 'prositvd188', 'prositvd193', 'prositvd195', 'prositvd196', 'prositvd198', 'prositvd200'])

In [74]:
all_results['prositvd42']

,date,duration
0,2022-11-21,0.000000
1,2022-11-22,4.600000
2,2022-11-23,0.983333
3,2022-11-24,68.716667
4,2022-11-25,0.000000
5,2022-11-26,0.000000
6,2022-11-27,0.000000
7,2022-11-28,0.000000
8,2022-11-29,0.000000
9,2022-11-30,0.800000
